In [23]:
import pandas as pd
import dask.dataframe as dd
from fuzzywuzzy import fuzz

In [2]:
raw_data = dd.read_csv('./Resultados11_nonull.csv', dtype='str',
                   assume_missing=True,
                   na_values=['', 'NA', 'nan'])
raw_data: pd.DataFrame = raw_data.compute()
raw_data

,PERIODO,ESTU_TIPODOCUMENTO,ESTU_CONSECUTIVO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_COD_DEPTO_UBICACION,...,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,DESEMP_INGLES,PUNT_INGLES,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_LECTURA_CRITICA,PUNT_GLOBAL
0,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,141016000305,241016000342,41,...,Si,Si,Si,B1,71,66,70,65,69,339
1,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,141016000305,241016000342,41,...,Si,Si,Si,B1,71,66,70,65,69,339
2,20194,TI,SB11201940562856,URBANO,N,A,TÉCNICO/ACADÉMICO,141396000175,141396000175,41,...,No,No,No,A-,46,43,31,40,43,199
3,20194,TI,SB11201940562856,URBANO,N,A,TÉCNICO/ACADÉMICO,141396000175,141396000175,41,...,No,No,No,A-,46,43,31,40,43,199
4,20162,TI,SB11201620084846,URBANO,N,A,TÉCNICO/ACADÉMICO,176001013310,176001013310,76,...,Si,Si,Si,A2,59,57,46,57,57,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155028,20221,CC,AC202210023501,URBANO,N,OTRO,ACADÉMICO,305001022682,305001022682,05,...,Si,Si,Si,A-,42,50,45,40,52,232
155029,20221,TI,AC202210003698,URBANO,S,B,ACADÉMICO,311001107513,311001107513,11,...,Si,Si,Si,B1,76,55,54,56,56,284
155030,20221,CC,AC202210034448,URBANO,N,A,ACADÉMICO,311001105928,311001105928,11,...,Si,Si,Si,A1,48,53,38,45,50,233
155031,20221,TI,AC202210006814,URBANO,S,B,TÉCNICO/ACADÉMICO,311001019568,311001019568,11,...,Si,Si,Si,B1,73,67,57,62,63,315


In [3]:
raw_data.columns

Index(['PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSO

In [4]:
data = raw_data.copy()

In [8]:
# Deberia ser de 33
display(len(data['COLE_DEPTO_UBICACION'].unique()))
data['COLE_DEPTO_UBICACION'].unique()

33

array(['HUILA', 'VALLE', 'ANTIOQUIA', 'TOLIMA', 'SANTANDER', 'BOLIVAR',
       'MAGDALENA', 'NARIÑO', 'CUNDINAMARCA', 'CASANARE', 'BOGOTÁ',
       'NORTE SANTANDER', 'CALDAS', 'BOYACA', 'CHOCO', 'CORDOBA', 'SUCRE',
       'CESAR', 'META', 'ATLANTICO', 'CAUCA', 'CAQUETA', 'RISARALDA',
       'LA GUAJIRA', 'QUINDIO', 'PUTUMAYO', 'GUAVIARE', 'AMAZONAS',
       'ARAUCA', 'VICHADA', 'SAN ANDRES', 'VAUPES', 'GUAINIA'],
      dtype=object)

In [11]:
data['COLE_MCPIO_UBICACION'].value_counts()

COLE_MCPIO_UBICACION
BOGOTÁ D.C.     575204
CALI            160472
BARRANQUILLA    107852
MEDELLIN        104285
MEDELLÍN         89364
                 ...  
YAVARATE            42
TARAPACÁ            38
SIPÍ                26
YAVARATÉ            26
SIPI                 9
Name: count, Length: 1396, dtype: int64

In [24]:
municipalities = data['COLE_MCPIO_UBICACION'].unique()

threshold: int = 70

enumerated_municipalities = enumerate(municipalities)

for i, current in enumerated_municipalities:
    for j, other in enumerated_municipalities:
        if i != j:
            similarity = fuzz.ratio(current, other)

            if similarity > threshold:
                display(f'High Similarity. | Current: {current} | Other: {other} | {similarity}%')

## @brief

Con lo anterior se puede concluir que no hay similaritudes relevantes entre los nombres de los municipios, asi que se puede proceder con
los siguientes pasos

In [22]:
data['COLE_NOMBRE_ESTABLECIMIENTO'].value_counts()

COLE_NOMBRE_ESTABLECIMIENTO
INSTITUCION EDUCATIVA SAN JOSE                    9512
INSTITUCION EDUCATIVA JORGE ELIECER GAITAN        8557
INSTITUCION EDUCATIVA SIMON BOLIVAR               8010
INSTITUCION EDUCATIVA ALFONSO LOPEZ PUMAREJO      7812
INSTITUCION EDUCATIVA FRANCISCO JOSE DE CALDAS    7466
                                                  ... 
CENTRO SOCIAL DE ESTUDIOS                            2
INST NIÑO JESUS DE PRAGA                             2
COLEGIO SEMILLA DE MOSTAZA                           1
CORPORACION EDUCATIVA Y TECNICA BOLIVARIANA          1
CENT ACAD DE EDUCACION PARA ADULTOS                  1
Name: count, Length: 7912, dtype: int64

In [32]:
institutions = data['COLE_NOMBRE_ESTABLECIMIENTO'].unique()

threshold: int = 80
enumerated_intitutions = enumerate(institutions)

for i, current in enumerated_intitutions:
    for j, other in enumerated_intitutions:
        if i != j:
            similarity = fuzz.ratio(current, other)

            if similarity > threshold:
                display(f'High Similarity. | Current: {current} | Other: {other} | {similarity}%')

'High Similarity. | Current: INSTITUCION EDUCATIVA JESUS MARIA AGUIRRE CHARRY | Other: INSTITUCION EDUCATIVA JESUS MARIA ORMAZA | 82%'

## @brief 

Con lo anterior tambien se puede notar que no parece que haya nombres de instituciones con errores gramaticales, asi que se puede continuar

In [33]:
data[['COLE_DEPTO_UBICACION', 'COLE_MCPIO_UBICACION', 'COLE_NOMBRE_ESTABLECIMIENTO']]

,COLE_DEPTO_UBICACION,COLE_MCPIO_UBICACION,COLE_NOMBRE_ESTABLECIMIENTO
0,HUILA,AIPE,INSTITUCION EDUCATIVA JESUS MARIA AGUIRRE CHARRY
1,HUILA,AIPE,INSTITUCION EDUCATIVA JESUS MARIA AGUIRRE CHARRY
2,HUILA,LA PLATA,INSTITUCION EDUCATIVA SAN SEBASTIAN
3,HUILA,LA PLATA,INSTITUCION EDUCATIVA SAN SEBASTIAN
4,VALLE,CALI,INSTITUCION EDUCATIVA POLITECNICO MUNICIPAL DE...
...,...,...,...
155028,ANTIOQUIA,MEDELLÍN,POLITÉCNICO MAYOR AGENCIA CRISTIANA DE SERVICI...
155029,BOGOTÁ,BOGOTÁ D.C.,COLEGIO VICTORIA
155030,BOGOTÁ,BOGOTÁ D.C.,CENT EDUC PROEDUCAR
155031,BOGOTÁ,BOGOTÁ D.C.,LIC FRANCES LOUIS PASTEUR
